## Solving puzzles with Python

See [2009 U.S. Puzzle Championship](http://wpc.puzzles.com/history/tests/ca9/uspc-09.pdf) for reference. PDF password is `barMd345`

#### 1. Battleships

Probably relevant paper: [Constraint Programming Models for Solitaire Battleships](https://pdfs.semanticscholar.org/f8a1/b0fd9d5b875549558973b2f972c72fd4dc83.pdf)

In [2]:
# ship pseudo-graphic:
# h - ltr head
# t - ltr tail
# H - up-down head
# T - up-down tail
# b - ltr body segment
# B - up-down body segment
# s - 1-unit submarine
# W - water (empty cell)'

In [93]:
from string import ascii_uppercase
from operator import eq
from collections import deque
from itertools import chain
from constraint import Problem, AllDifferentConstraint

grid_size = 10 + 2
possible_values = range((grid_size) ^ 2)
visibility = {'T': 1, 'H': 1, 'B': 1, 'S': 1, 'W': 0}
row_visible_parts = [0] + [4, 0, 2, 1, 1, 2, 0, 5, 0, 5] + [0]
col_visible_parts = [0] + [2, 2, 1, 1, 1, 2, 2, 1, 6, 2] + [0]
known_parts = {(0, 0): 's', '20': 'W', '92': 'B', '97': 'H'}
fleet_size = 10

b2 = list(combinations_with_replacement(['hbbt', 'HBBT'], 1))
b1 = list(combinations_with_replacement(['hbt', 'HBT'], 2))
b0 = list(combinations_with_replacement(['ht', 'HT'], 3))
subs = ['s', 's', 's', 's']

fleet_permutations = [chain(x, y, z, subs) for x in b2 for y in b1 for z in b0]

edging_map = {None: {'rotate': 0, 'wcount': 0},
              'ul': {'rotate': 0, 'wcount': 0},
              'u':  {'rotate': 0, 'wcount': 0},
              'ur': {'rotate': 0, 'wcount': 0},
              'r':  {'rotate': 0, 'wcount': 0},
              'br': {'rotate': 0, 'wcount': 0},
              'b':  {'rotate': 0, 'wcount': 0},
              'bl': {'rotate': 0, 'wcount': 0},
              'l':  {'rotate': 0, 'wcount': 0}}

def space_constraint(*box, ship='hbbt', edge=None):
    '''
    W W W W W W
    W h b b t W
    W W W W W W
    '''
    d = deque(box)
    d.reverse()
    d.rotate(len(ship) + 3)
    return ''.join([d.pop() for _ in range(len(ship))]) == ship and \
           d.count('w') == len(box) - len(ship)

for fleet in fleet_permutations:
    p = Problem()
    p.addVariables(range(fleet_size), possible_values)
    p.addConstraint(AllDifferentConstraint(), range(fleet_size))
#     print(p.getSolution())
# TODO: implement more constraints from  the paper

#### 2. Sudoku

In [11]:
# credit: https://simplapi.wordpress.com/2012/11/02/python-constraint-and-sudoku/

from constraint import Problem, InSetConstraint, AllDifferentConstraint
import math
 
def solveSudoku(size = 9, originalGame = None):
    """ Solving Sudoku of any size """
    sudoku = Problem()
 
    #Defining size of row/col
    rows = range(size)
    cols = range(size)
 
    #Creating board
    board = [(row, col) for row in rows for col in cols]
    #Defining game variable, a single range will be enough
    sudoku.addVariables(board, range(1, size + 1))
 
    #Row set
    rowSet = [list(zip([el] * len(cols), cols)) for el in rows]
    colSet = [list(zip(rows, [el] * len(rows))) for el in cols]
  
    #The original board is not empty, we add that constraint to the list of constraint
    if originalGame is not None:
        for i in range(0, size):
            for j in range(0, size):
                #Getting the value of the current game
                o = originalGame[i][j]
                #We apply constraint when the number is set only
                if o > 0:
                    #We get the associated tuple
                    t = (rows[i],cols[j])
                    #We set a basic equal constraint rule to force the system to keep that variable at that place
                    sudoku.addConstraint(lambda var, val=o: var == val, (t,))
 
    #The constraint are like that : and each row, and each columns, got same final compute value
 
    for row in rowSet:
        sudoku.addConstraint(AllDifferentConstraint(), row)
    for col in colSet:
        sudoku.addConstraint(AllDifferentConstraint(), col)
 
    #Every sqrt(size) (3x3 box constraint) got same sum
    sqSize = int(math.floor(math.sqrt(size)))
 
    #xrange allow to define a step, here sq (wich is sq = 3 in 9x9 sudoku)
    for i in range(0,size,sqSize):
        for j in range(0,size,sqSize):
            #Computing the list of tuple linked to that box
            box = []
            for k in range(0, sqSize):
                for l in range(0, sqSize):
                    #The tuple i+k, j+l is inside that box
                    box.append( (i+k, j+l) )
            #Compute is done, now we can add the constraint for that box
            sudoku.addConstraint(AllDifferentConstraint(), box)
 
    #Computing and returning final result
    return sudoku.getSolution()
 
 

rg = 9
initValue = [[0, 1, 0, 5, 0, 0, 0, 8, 0],
             [0, 2, 3, 4, 0, 0, 7, 0, 0],
             [0, 0, 0, 0, 0, 6, 0, 0, 0],
             [0, 0, 8, 1, 0, 0, 0, 0, 5],
             [0, 7, 0, 0, 0, 0, 0, 4, 0],
             [6, 0, 0, 0, 0, 3, 2, 0, 0],
             [0, 0, 0, 6, 0, 0, 0, 0, 0],
             [0, 0, 7, 0, 0, 4, 3, 2, 0],
             [0, 8, 0, 0, 0, 5, 0, 1, 0]]

res = solveSudoku(rg, initValue)
if res is not None:
#     for i in range(0, rg):
#         for j in range(0, rg):
#             print(res[i, j], end='')
#         print()
#     print()
    
    print([res[6, i] for i in range(rg)])
    print([res[i, 6] for i in range(rg)])
else:
    print("No result to show")

[3, 4, 1, 6, 2, 8, 5, 9, 7]
[4, 7, 1, 9, 8, 2, 5, 3, 6]


#### 3. Missing Operation KenKen

In [12]:
from constraint import Problem, AllDifferentConstraint
from functools import partial, reduce
from operator import mul

size = 6

kenken = Problem()

#Defining size of row/col
rows = range(size)
cols = range(size)

#Creating board
board = [(row, col) for row in rows for col in cols]
#Defining game variable, a single range will be enough
kenken.addVariables(board, range(1, size + 1))

#Row set
rowSet = [list(zip([el] * len(cols), cols)) for el in rows]
colSet = [list(zip(rows, [el] * len(rows))) for el in cols]

for row in rowSet:
    kenken.addConstraint(AllDifferentConstraint(), row)
for col in colSet:
    kenken.addConstraint(AllDifferentConstraint(), col)

regions = [  (4,  [(0, 0), (0, 1)]),
             (40, [(0, 2), (0, 3), (1, 3)]),
             (3,  [(0, 4), (0, 5)]),
             (12, [(1, 0), (1, 1), (1, 2)]),
             (3,  [(1, 4), (1, 5), (2, 5)]),
             (15, [(2, 0), (3, 0), (3, 1)]),
             (2,  [(2, 1)]),
             (3,  [(2, 2), (3, 2)]),
             (10, [(2, 3), (2, 4)]),
             (21, [(3, 3), (3, 4), (3, 5), (4, 4), (4, 5)]),
             (6,  [(4, 0), (4, 1), (4, 2)]),
             (20, [(4, 3), (5, 3), (5, 2)]),
             (2,  [(5, 0), (5, 1)]),
             (2,  [(5, 4), (5, 5)])]
    

def func(*cells, value=None):
    if len(cells) == 1:
        return cells[0] == value
    elif len(cells) == 2:
        return any([sum(cells) == value,
                    cells[0] - cells[1] == value,
                    cells[1] - cells[0] == value,
                    mul(*cells) == value,
                    cells[0] / cells[1] == value,
                    cells[1] / cells[0] == value])
    else:
        return any([sum(cells) == value,
                    reduce(mul, cells) == value])

for val, region in regions:
    kenken.addConstraint(partial(func, value=val), region)
    

res = kenken.getSolution()
if res is not None:
#     for i in range(0, size):
#         for j in range(0, size):
#             print(res[i, j], end='')
#         print()
#     print()
    print([res[4, i] for i in range(size)])
    print([res[i, 4] for i in range(size)])
else:
    print("No solution")

[3, 1, 2, 4, 6, 5]
[3, 1, 4, 5, 6, 2]


#### 4. Sum Thing

In [16]:
from constraint import Problem, AllDifferentConstraint, ExactSumConstraint

size = 10
line_sum = 13
board = range(size)
lines = [[0, 1, 7], [0, 5, 8], [0, 2, 4], [1, 2, 3], [1, 5, 9],
         [2, 6, 7], [3, 5, 7], [3, 4, 9], [4, 5, 6], [6, 8, 9]]

p = Problem()
p.addVariables(board, range(size))
p.addConstraint(AllDifferentConstraint(), board)
for line in lines:
    p.addConstraint(ExactSumConstraint(line_sum), line)

res = p.getSolution()
if res is not None:
    print(', '.join([str(res[i]) for i in board]))
else:
    print("No solution")

0, 6, 5, 2, 8, 4, 1, 7, 9, 3


#### 7. Writer's Block

In [1]:
from constraint import Problem, AllDifferentConstraint
from functools import partial

names = [
    'ALBEE', 'ALBOM', 'ALGER', 'BLAKE', 'FROST', 'GRIMM', 'MAMET', 'PLATH', 'TYLER', 'WELTY',
    'ASIMOV', 'BRONTE', 'CAPOTE', 'CHABON', 'HORACE', 'JEWETT', 'KILMER', 'MAILER', 'MILLER',
    'MILTON', 'ONEILL', 'FAULKNER', 'MACLEISH', 'MCCARTHY', 'MELVILLE', 'MICHENER', 'HARPERLEE',
    'HEMINGWAY', 'JAMESAGEE', 'JKROWLING', 'NEILSIMON'
]

places_count = 27
places = range(places_count)

crisscross = Problem()
crisscross.addVariables(places, names)

crisscross.addConstraint(AllDifferentConstraint(), places)

place_sizes = {0:5, 1:5, 2:9, 3:6, 4:6, 5:8, 6:5, 7:8, 8:6, 9:6,
               10:8, 11:9, 12:9, 13:5, 14:6, 15:6, 16:5, 17:6, 18:6,
               19:6, 20:5, 21:8, 22:5, 23:5, 24:5, 25:6, 26:9}

for place, size in place_sizes.items():
    crisscross.addConstraint(lambda x, s=size: len(x) == s, (place, ))

# place1, index1, place2, index2
intersections = [
    (0, 0, 17, 0), (4, 0, 17, 2), (12, 3, 17, 5), (23, 1, 4, 5), (23, 4, 12, 8), (21, 1, 12, 5),
    (12, 0, 15, 0), (16, 4, 25, 0),  (19, 2, 24, 1), (21, 6, 24, 3), (19, 4, 26, 7),
    (26, 1, 11, 7), (11, 0, 10, 6), (11, 3, 18, 2), (18, 5, 12, 3), (12, 5, 21, 1),
    (22, 1, 11, 5), (11, 7, 26, 1), (9, 1, 2, 3), (9, 3, 8, 0), (9, 5, 10, 4), (13, 0, 2, 6),
    (8, 3, 13, 2), (10, 7, 13, 4), (7, 0, 6, 4), (1, 2, 6, 0), (10, 0, 5, 0), (10, 2, 7, 2),
    (14, 0, 3, 0), (14, 2, 5, 2), (14, 4, 7, 4), (20, 0, 3, 3), (20, 2, 5, 5), (20, 4, 7, 7)
]

def intersection_func(p1, p2, i1=None, i2=None):
    try:
        return p1[i1] == p2[i2]
    except IndexError:
        return False

for place1, idx1, place2, idx2 in intersections:
    crisscross.addConstraint(partial(intersection_func, i1=idx1, i2=idx2), (place1, place2))

res = crisscross.getSolution()

# print(res or "No solution")

set(names) - set(res.values())

{'ALBOM', 'ASIMOV', 'JKROWLING', 'MACLEISH'}

#### 9. Coordinate Pairs

In [2]:
from itertools import combinations, chain
from math import sqrt, pow
from collections import defaultdict
from decimal import Decimal


coords = {'A':(1,6), 'B':(2,6), 'C':(2,5), 'D':(5,5), 'E':(0,4), 'F':(2,4), 'G':(3,4),
          'H':(4,4), 'I':(6,4), 'J':(5,3), 'K':(0,2), 'L':(0,1), 'M':(1,1), 'N':(4,0)}

len_coords = len(coords)
len_segment_vars = int(len(coords.keys())/2)

def get_length(p1, p2):
    return Decimal(sqrt(pow((p2[0] - p1[0]), 2) + \
                        pow((p2[1] - p1[1]), 2)))

segments_length_map = {f'{p1}{p2}':get_length(coords[p1], coords[p2]) \
                       for p1, p2 in list(combinations(coords.keys(), 2))}

segments_length_reverse_map = defaultdict(list)

for k, v in segments_length_map.items():
    segments_length_reverse_map[v].append(k)

def get_letter_sequence(length_pair):
    return list(chain(*map(segments_length_reverse_map.get, length_pair)))

def has_all_unique_letters(segs):
    seen = set(''.join(segs))
    return len(seen) == len_coords

for length_pair in combinations(segments_length_reverse_map.keys(), 2):
    segs = get_letter_sequence(length_pair)
    if has_all_unique_letters(segs):
        for comb in filter(has_all_unique_letters, combinations(segs, len_segment_vars)):
            print(sorted(comb))

['AE', 'BI', 'CL', 'DG', 'FN', 'HK', 'JM']


#### 10. Triangular Skyscrapers

In [3]:
from constraint import Problem, AllDifferentConstraint
from functools import partial

cells = range(32)
values = range(1, 9)

p = Problem()
p.addVariables(cells, values)

p.addConstraint(lambda x: x==1, (1, ))

rows = [list(range(8*x, 8*(x+1))) for x in range(4)]
rows_visibility = [3, 4, 2, 5]
cols = [[31, 30, 17, 16, 15, 14, 1, 0],
        [3, 2, 13, 12, 19, 18, 29, 28],
        [27, 26, 21, 20, 11, 10, 5, 4],
        [7, 6, 9, 8, 23, 22, 25, 24]]
cols_visibility = [4, 3, 6, 2]

cell_types = {'◤': [0, 4, 9, 13, 16, 20, 25, 29],
              '◢': [1, 5, 8, 12, 21, 17, 24, 28],
              '◣': [2, 6, 11, 15, 18, 22, 27, 31],
              '◥': [3, 7, 10, 14, 19, 23, 26, 30]}
for groups in cell_types.values():
    p.addConstraint(AllDifferentConstraint(), groups)

def visible(*cells, val=0):
    highest = 0
    visible = 0
    for c in cells:
        if c > highest:
            highest = c
            visible += 1
    return visible == val

for line, vis in zip(chain(rows, cols), chain(rows_visibility, cols_visibility)):
    p.addConstraint(AllDifferentConstraint(), line)
    p.addConstraint(partial(visible, val=vis), line)

sol = p.getSolution()
print([sol[x] for x in rows[1][::-1]], [sol[x] for x in cols[1][::-1]])

[2, 8, 1, 6, 7, 3, 5, 4] [7, 8, 4, 2, 6, 1, 3, 5]


#### 11. Window Pain

In [91]:
from itertools import islice, combinations, starmap

step = 1

x_steps = [0, 2*step, step, step, step, 3*step, 2*step, step, step, step]
y_steps = [0, step, step, 3*step, step, step, 2*step, step, step]

points = [(sum(islice(x_steps, x+1)),
           sum(islice(y_steps, y+1))) \
          for x in range(len(x_steps)) \
          for y in range(len(y_steps))]

diags = combinations(points, 2)

def is_rectangle(p1, p2):
    return abs(p1[0] - p2[0]) == abs(p1[1] - p2[1])

sum(starmap(is_rectangle, diags)) // 2

152

#### 12. Masyu

In [94]:
size = 17
grid = range(size * size)

class Grid:
    pass

path_list = ['│', '─', '┌', '┐', '└', '┘']

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.in_point = None
        self.out_point = None
        self.visited = False
        self.path = None
        self.is_start_point = False
        
    def is_visited(self):
        return self.visited
    
    def visit(self, in_point=None):
        self.in_point = in_point
        self.visited = True
    
    def can_move_down(self):
        return self.y+1 < size and not Grid.point_at(self.x, self.y+1).visited
    
    def can_move_up(self):
        return self.y-1 >= 0 and not Grid.point_at(self.x, self.y-1).visited
    
    def can_move_left(self):
        return self.x-1 >= 0 and not Grid.point_at(self.x-1, self.y).visited
    
    def can_move_right(self):
        return self.x+1 < size and not Grid.point_at(self.x+1, self.y).visited
    
    def next_moves(self):
        moves = []
        if self.path == '│':
            # downward movement
            if self.in_point.y < self.y:
                if self.can_move_down():
                    # TODO: check Circle color logic
                    moves.append(Grid.point_at(self.x, self.y+1))
            else:
                if self.can_move_up():
                    #TODO: check Circle color logic
                    moves.append(Grid.point_at(self.x, self.y-1))
        elif self.path == '─':
            # left-to-right movement
            if self.in_point.x < self.x:
                if self.can_move_right():
                    moves.append
                

        
class Circle(Point):
    def __init__(self, x, y, color=None, letter=None):
        super(Circle, self).__init__(x, y)
        self.color = color
        self.letter = letter
    
    
        
    
    
    